In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Data

In [29]:
# Load the dataframes
train_data = pd.read_csv("data/train.csv")
test_data=pd.read_csv("data/test.csv")

# Preview the dataframes
display(train_data.head(10))
display(test_data.head(10))

,Sequence,Active
0,DKWL,0
1,FCHN,0
2,KDQP,0
3,FNWI,0
4,NKRM,0
5,IWHA,0
6,GEFV,0
7,KARS,0
8,IMGW,0
9,KYRK,0


,Sequence
0,HWFK
1,MWPW
2,ALDV
3,NTLG
4,LHYY
5,AFGM
6,TPNY
7,EAKD
8,NDKE
9,GFIS


,Active
count,112000.000000
mean,0.037616
std,0.190267
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [3]:
def create_char_df(df):
    """Create a dataframe that contains four columns (one for each char in 'Sequence')."""

    # Split sequences into characters
    first = []
    second = []
    third = []
    fourth = []

    for _, row in df.iterrows():
        ch_1, ch_2, ch_3, ch_4 = row["Sequence"]

        first.append(ch_1)
        second.append(ch_2)
        third.append(ch_3)
        fourth.append(ch_4)

    df_dict = {
        "first": first,
        "second": second,
        "third": third,
        "fourth": fourth
    }
    return pd.DataFrame(df_dict)

X_df = create_char_df(train_data)
X_test_df = create_char_df(test_data)

display(X_df.head(5))
display(X_test_df.head(5))

,first,second,third,fourth
0,D,K,W,L
1,F,C,H,N
2,K,D,Q,P
3,F,N,W,I
4,N,K,R,M


,first,second,third,fourth
0,H,W,F,K
1,M,W,P,W
2,A,L,D,V
3,N,T,L,G
4,L,H,Y,Y


## OneHot Encoding

In [4]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_df)

display(enc.categories_)

# Ensure the onehots are the same for each letter
assert((enc.categories_[0] == enc.categories_[1]).all() and (enc.categories_[1] == enc.categories_[2]).all() and (enc.categories_[2] == enc.categories_[3]).all())

[array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object)]

# Training

In [23]:
from sklearn.model_selection import train_test_split

X = enc.transform(X_df).toarray().reshape(-1, 4, 20)
y = train_data["Active"].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=1337)

# Create training np arrays
#X_train = enc.transform(X_df).toarray().reshape(-1, 4, 20)
#y_train = train_data["Active"].values

In [32]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

#clf = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(15,), batch_size=128, verbose=True), verbose=True)
#clf.fit(X_train, y_train)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from sklearn.utils.class_weight import compute_class_weight

class_weight = compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {
    0: class_weight[0],
    1: class_weight[1]
}

model_lstm = Sequential()
model_lstm.add(
    Bidirectional(LSTM(4, activation='relu', input_shape=(128, 20)))
)
model_lstm.add(Dense(1, activation = 'sigmoid'))
model_lstm.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

model_lstm.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_val, y_val), class_weight=class_weights)

Epoch 1/30
832/832 [==============================] - 3s 4ms/step - loss: 0.4495 - accuracy: 0.8423 - val_loss: 0.3261 - val_accuracy: 0.8657
Epoch 2/30
832/832 [==============================] - 3s 3ms/step - loss: 0.2619 - accuracy: 0.8863 - val_loss: 0.2460 - val_accuracy: 0.9014
Epoch 3/30
832/832 [==============================] - 3s 3ms/step - loss: 0.2191 - accuracy: 0.9036 - val_loss: 0.2494 - val_accuracy: 0.8989
Epoch 4/30
832/832 [==============================] - 3s 3ms/step - loss: 0.1966 - accuracy: 0.9128 - val_loss: 0.2399 - val_accuracy: 0.9050
Epoch 5/30
832/832 [==============================] - 3s 3ms/step - loss: 0.1793 - accuracy: 0.9203 - val_loss: 0.2235 - val_accuracy: 0.9120
Epoch 6/30
832/832 [==============================] - 3s 3ms/step - loss: 0.1665 - accuracy: 0.9258 - val_loss: 0.2025 - val_accuracy: 0.9209
Epoch 7/30
832/832 [==============================] - 3s 3ms/step - loss: 0.1565 - accuracy: 0.9298 - val_loss: 0.1938 - val_accuracy: 0.9257
Epoch 

In [25]:
from sklearn.metrics import f1_score

y_pred = model_lstm.predict(X_val)
f1_score(y_val, y_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [33]:
X_test = enc.transform(X_test_df).toarray().reshape(-1, 4, 20)

In [34]:
y_test=model_lstm.predict(X_test)

In [35]:
np.savetxt("submission.txt", y_test, fmt="%d")

In [36]:
display(y_test[0:20])

array([[2.4926662e-04],
       [5.6092849e-06],
       [4.2979322e-05],
       [3.3139300e-01],
       [1.8942479e-05],
       [9.9990606e-01],
       [3.8763924e-06],
       [4.5645356e-02],
       [4.8471935e-07],
       [4.3003428e-07],
       [5.5068354e-05],
       [7.2101265e-08],
       [1.2124777e-03],
       [1.7029690e-08],
       [7.4565446e-06],
       [9.9670923e-01],
       [3.1134486e-04],
       [9.9363834e-01],
       [9.2260234e-05],
       [2.1108985e-04]], dtype=float32)